# Perceptron



In [1]:
import numpy as np
np.seterr('ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

# Perceptron Network

In [2]:
# Here I am building a normal forward propogation for a perceptron layer
# and the various function i would use.

In [3]:
#############################################
## Declaring Training Data        ############
#############################################
X_train = np.array([[0,0],[1,0],[0,1],[1,1]])
Y_train = np.array([[1],[0],[0],[0]])



In [4]:
def error(pred,Y):
    return (Y - pred)
    

In [5]:
from MultiLayerPerceptron import MultiLayerPerceptron

In [6]:
def accuracy(model , X , Y):
    pred,_ = model.forward(X)
    pred =pred > 0.5
    acc = np.sum(pred == Y)
    acc = float(acc) / Y.shape[0]
    return acc

In [7]:
# Declare a neuron with shape of weights as [shape_of_input,1]
model = MultiLayerPerceptron([2,1],['tanh'])

<function tanh at 0x7fc28f6f6598>
(2, 1) (1,)


In [8]:
# testing the model outputs
pred , _ = model.forward(X_train)
# print(np.sum((pred > 0.5)== Y_train) / Y_train.shape[0])
# accuracy(model , X_train,Y_train)
# X_train[0]

In [9]:
# Now we can train the model by iteratively on each datapoint.
from Loss import Loss
layer_list = [2 ,3, 1]
activation_list = ['sigmoid','sigmoid']
model = MultiLayerPerceptron(layer_list,activation_list)
loss_m = Loss()

def train(model ,loss_m,X_train , Y_train,epochs,alpha =0.1):
    for i in range(epochs):
        act , cache = model.forward(X_train)
        loss,d_back= loss_m.sq_loss(act,Y_train)
        acc = accuracy(model ,X_train,Y_train)
        print("The training loss at {}th epoch : {}  Training Accuracy:{}".format(i , loss , acc))
        model.update_gradient(cache,d_back,alpha)        

train(model , loss_m , X_train,Y_train , 50,alpha = 0.1)
    

<function sigmoid at 0x7fc28f6f6400>
(2, 3) (3,)
<function sigmoid at 0x7fc28f6f6400>
(3, 1) (1,)
The training loss at 0th epoch : 0.7722248927824626  Training Accuracy:0.25
The training loss at 1th epoch : 0.74356641905686  Training Accuracy:0.25
The training loss at 2th epoch : 0.7149791382726095  Training Accuracy:0.25
The training loss at 3th epoch : 0.6866967799514747  Training Accuracy:0.5
The training loss at 4th epoch : 0.6589600727045021  Training Accuracy:0.5
The training loss at 5th epoch : 0.6320062078443989  Training Accuracy:0.5
The training loss at 6th epoch : 0.6060575321474214  Training Accuracy:0.5
The training loss at 7th epoch : 0.581310724230354  Training Accuracy:0.5
The training loss at 8th epoch : 0.5579277441099357  Training Accuracy:0.5
The training loss at 9th epoch : 0.5360296074096995  Training Accuracy:0.5
The training loss at 10th epoch : 0.5156935757785212  Training Accuracy:0.5
The training loss at 11th epoch : 0.4969537966580385  Training Accuracy:0.5


In [10]:
# Now we can predict the values for unseen data or trained data also
# We can also calculate the accuracy of the model we have trained
accuracy(model , X_train,Y_train)

0.75

# N Bit XOR

Now Lets try working with just a little better data. A n XOR operator. So lets create the dataset for n bit xor.

We would have atmost 2^n data point in this type of data set.BUt we would limit our dataset to a 1000 data points
whichever is smaller.

Then we can divide into training and test set

In [12]:

n = 10
max_datapoint = 10000
datapoints = min(pow(2,n) , max_datapoint)

X = np.zeros((datapoints , n) , dtype=np.int32)
Y = np.zeros((datapoints , 1), dtype=np.int32)

for i in range(datapoints):
    tmp = i
    y_tmp = 0
    for j in range(n-1 , -1 , -1):
        X[i,j] = tmp&1
        y_tmp = y_tmp^X[i,j]
        tmp = tmp>>1
    Y[i] = y_tmp
        
        

In [13]:
# for sanity check lets print one example
ind = 11
print(X[ind] , Y[ind])

[0 0 0 0 0 0 1 0 1 1] [1]


In [14]:
# Lets divide the set in training and testing
div = 0.9
train_n = int(div * datapoints)
X_train = X[:train_n]
Y_train = Y[:train_n]

X_test = X[train_n:]
Y_test = Y[train_n:]
Y_test.shape

(103, 1)

In [17]:
layer_list = [n,8,2,1]
activation_list = ['sigmoid','linear','tanh']

loss_m = Loss()


model = MultiLayerPerceptron(layer_list,activation_list)

<function sigmoid at 0x7fc28f6f6400>
(10, 8) (8,)
<function linear at 0x7fc28f6f66a8>
(8, 2) (2,)
<function tanh at 0x7fc28f6f6598>
(2, 1) (1,)


In [19]:
train(model ,loss_m,X_train,Y_train , 2000,alpha = 0.01)

The training loss at 0th epoch : 115.12318449879041  Training Accuracy:0.501628664495114
The training loss at 1th epoch : 115.12318236276869  Training Accuracy:0.501628664495114
The training loss at 2th epoch : 115.12318028288092  Training Accuracy:0.500542888165038
The training loss at 3th epoch : 115.12317825780892  Training Accuracy:0.500542888165038
The training loss at 4th epoch : 115.12317628709329  Training Accuracy:0.500542888165038
The training loss at 5th epoch : 115.12317443425857  Training Accuracy:0.500542888165038
The training loss at 6th epoch : 115.12319718536631  Training Accuracy:0.501628664495114
The training loss at 7th epoch : 115.13109089369168  Training Accuracy:0.500542888165038
The training loss at 8th epoch : 117.89300815101502  Training Accuracy:0.499457111834962
The training loss at 9th epoch : 225.45912602675637  Training Accuracy:0.500542888165038
The training loss at 10th epoch : 220.30682906224257  Training Accuracy:0.500542888165038
The training loss at

The training loss at 127th epoch : 229.99999964298343  Training Accuracy:0.500542888165038
The training loss at 128th epoch : 229.9999996429834  Training Accuracy:0.500542888165038
The training loss at 129th epoch : 229.99999964298337  Training Accuracy:0.500542888165038
The training loss at 130th epoch : 229.9999996429833  Training Accuracy:0.500542888165038
The training loss at 131th epoch : 229.99999964298325  Training Accuracy:0.500542888165038
The training loss at 132th epoch : 229.99999964298323  Training Accuracy:0.500542888165038
The training loss at 133th epoch : 229.9999996429832  Training Accuracy:0.500542888165038
The training loss at 134th epoch : 229.99999964298314  Training Accuracy:0.500542888165038
The training loss at 135th epoch : 229.99999964298308  Training Accuracy:0.500542888165038
The training loss at 136th epoch : 229.99999964298303  Training Accuracy:0.500542888165038
The training loss at 137th epoch : 229.999999642983  Training Accuracy:0.500542888165038
The 

The training loss at 262th epoch : 229.9999996429776  Training Accuracy:0.500542888165038
The training loss at 263th epoch : 229.99999964297754  Training Accuracy:0.500542888165038
The training loss at 264th epoch : 229.9999996429775  Training Accuracy:0.500542888165038
The training loss at 265th epoch : 229.99999964297746  Training Accuracy:0.500542888165038
The training loss at 266th epoch : 229.99999964297743  Training Accuracy:0.500542888165038
The training loss at 267th epoch : 229.99999964297737  Training Accuracy:0.500542888165038
The training loss at 268th epoch : 229.99999964297734  Training Accuracy:0.500542888165038
The training loss at 269th epoch : 229.9999996429773  Training Accuracy:0.500542888165038
The training loss at 270th epoch : 229.99999964297723  Training Accuracy:0.500542888165038
The training loss at 271th epoch : 229.99999964297723  Training Accuracy:0.500542888165038
The training loss at 272th epoch : 229.99999964297717  Training Accuracy:0.500542888165038
Th

The training loss at 400th epoch : 229.99999964297163  Training Accuracy:0.500542888165038
The training loss at 401th epoch : 229.99999964297157  Training Accuracy:0.500542888165038
The training loss at 402th epoch : 229.99999964297155  Training Accuracy:0.500542888165038
The training loss at 403th epoch : 229.99999964297152  Training Accuracy:0.500542888165038
The training loss at 404th epoch : 229.99999964297143  Training Accuracy:0.500542888165038
The training loss at 405th epoch : 229.9999996429714  Training Accuracy:0.500542888165038
The training loss at 406th epoch : 229.99999964297135  Training Accuracy:0.500542888165038
The training loss at 407th epoch : 229.99999964297132  Training Accuracy:0.500542888165038
The training loss at 408th epoch : 229.9999996429713  Training Accuracy:0.500542888165038
The training loss at 409th epoch : 229.99999964297123  Training Accuracy:0.500542888165038
The training loss at 410th epoch : 229.9999996429712  Training Accuracy:0.500542888165038
Th

The training loss at 544th epoch : 229.9999996429654  Training Accuracy:0.500542888165038
The training loss at 545th epoch : 229.99999964296535  Training Accuracy:0.500542888165038
The training loss at 546th epoch : 229.99999964296532  Training Accuracy:0.500542888165038
The training loss at 547th epoch : 229.99999964296524  Training Accuracy:0.500542888165038
The training loss at 548th epoch : 229.99999964296524  Training Accuracy:0.500542888165038
The training loss at 549th epoch : 229.99999964296518  Training Accuracy:0.500542888165038
The training loss at 550th epoch : 229.99999964296512  Training Accuracy:0.500542888165038
The training loss at 551th epoch : 229.9999996429651  Training Accuracy:0.500542888165038
The training loss at 552th epoch : 229.99999964296507  Training Accuracy:0.500542888165038
The training loss at 553th epoch : 229.999999642965  Training Accuracy:0.500542888165038
The training loss at 554th epoch : 229.99999964296495  Training Accuracy:0.500542888165038
The

...

The training loss at 1402th epoch : 229.99999964292826  Training Accuracy:0.500542888165038
The training loss at 1403th epoch : 229.99999964292823  Training Accuracy:0.500542888165038
The training loss at 1404th epoch : 229.99999964292817  Training Accuracy:0.500542888165038
The training loss at 1405th epoch : 229.99999964292815  Training Accuracy:0.500542888165038
The training loss at 1406th epoch : 229.9999996429281  Training Accuracy:0.500542888165038
The training loss at 1407th epoch : 229.99999964292806  Training Accuracy:0.500542888165038
The training loss at 1408th epoch : 229.999999642928  Training Accuracy:0.500542888165038
The training loss at 1409th epoch : 229.99999964292795  Training Accuracy:0.500542888165038
The training loss at 1410th epoch : 229.9999996429279  Training Accuracy:0.500542888165038
The training loss at 1411th epoch : 229.99999964292786  Training Accuracy:0.500542888165038
The training loss at 1412th epoch : 229.99999964292783  Training Accuracy:0.50054288

The training loss at 1541th epoch : 229.99999964292223  Training Accuracy:0.500542888165038
The training loss at 1542th epoch : 229.9999996429222  Training Accuracy:0.500542888165038
The training loss at 1543th epoch : 229.99999964292215  Training Accuracy:0.500542888165038
The training loss at 1544th epoch : 229.9999996429221  Training Accuracy:0.500542888165038
The training loss at 1545th epoch : 229.99999964292206  Training Accuracy:0.500542888165038
The training loss at 1546th epoch : 229.99999964292203  Training Accuracy:0.500542888165038
The training loss at 1547th epoch : 229.99999964292198  Training Accuracy:0.500542888165038
The training loss at 1548th epoch : 229.99999964292192  Training Accuracy:0.500542888165038
The training loss at 1549th epoch : 229.9999996429219  Training Accuracy:0.500542888165038
The training loss at 1550th epoch : 229.99999964292186  Training Accuracy:0.500542888165038
The training loss at 1551th epoch : 229.9999996429218  Training Accuracy:0.50054288

The training loss at 1675th epoch : 229.99999964291646  Training Accuracy:0.500542888165038
The training loss at 1676th epoch : 229.9999996429164  Training Accuracy:0.500542888165038
The training loss at 1677th epoch : 229.99999964291635  Training Accuracy:0.500542888165038
The training loss at 1678th epoch : 229.9999996429163  Training Accuracy:0.500542888165038
The training loss at 1679th epoch : 229.99999964291626  Training Accuracy:0.500542888165038
The training loss at 1680th epoch : 229.99999964291624  Training Accuracy:0.500542888165038
The training loss at 1681th epoch : 229.99999964291618  Training Accuracy:0.500542888165038
The training loss at 1682th epoch : 229.99999964291612  Training Accuracy:0.500542888165038
The training loss at 1683th epoch : 229.9999996429161  Training Accuracy:0.500542888165038
The training loss at 1684th epoch : 229.99999964291604  Training Accuracy:0.500542888165038
The training loss at 1685th epoch : 229.999999642916  Training Accuracy:0.500542888

The training loss at 1771th epoch : 229.9999996429123  Training Accuracy:0.500542888165038
The training loss at 1772th epoch : 229.99999964291223  Training Accuracy:0.500542888165038
The training loss at 1773th epoch : 229.9999996429122  Training Accuracy:0.500542888165038
The training loss at 1774th epoch : 229.99999964291214  Training Accuracy:0.500542888165038
The training loss at 1775th epoch : 229.99999964291212  Training Accuracy:0.500542888165038
The training loss at 1776th epoch : 229.99999964291206  Training Accuracy:0.500542888165038
The training loss at 1777th epoch : 229.99999964291203  Training Accuracy:0.500542888165038
The training loss at 1778th epoch : 229.99999964291197  Training Accuracy:0.500542888165038
The training loss at 1779th epoch : 229.99999964291192  Training Accuracy:0.500542888165038
The training loss at 1780th epoch : 229.9999996429119  Training Accuracy:0.500542888165038
The training loss at 1781th epoch : 229.99999964291186  Training Accuracy:0.5005428

The training loss at 1962th epoch : 229.99999964290402  Training Accuracy:0.500542888165038
The training loss at 1963th epoch : 229.99999964290396  Training Accuracy:0.500542888165038
The training loss at 1964th epoch : 229.9999996429039  Training Accuracy:0.500542888165038
The training loss at 1965th epoch : 229.99999964290387  Training Accuracy:0.500542888165038
The training loss at 1966th epoch : 229.99999964290384  Training Accuracy:0.500542888165038
The training loss at 1967th epoch : 229.9999996429038  Training Accuracy:0.500542888165038
The training loss at 1968th epoch : 229.99999964290373  Training Accuracy:0.500542888165038
The training loss at 1969th epoch : 229.9999996429037  Training Accuracy:0.500542888165038
The training loss at 1970th epoch : 229.99999964290367  Training Accuracy:0.500542888165038
The training loss at 1971th epoch : 229.99999964290362  Training Accuracy:0.500542888165038
The training loss at 1972th epoch : 229.9999996429036  Training Accuracy:0.50054288

In [20]:
print("Training accuracy:{}".format(accuracy(model,X_train, Y_train)))

print("Test accuracy:{}".format(accuracy(model,X_test, Y_test)))


Training accuracy:0.500542888165038
Test accuracy:0.49514563106796117
